# ADAPTmap_genotypeTOP_20161201
Try to describe `ADAPTmap_genotypeTOP_20161201.zip` data files. This dataset is paired with `ADAPTmap_phenotype_20161201.zip` dataset

In [1]:
import pandas as pd
from plinkio import plinkfile

from src.features.smarterdb import global_connection, Dataset
from src.data.common import pandas_open

In [2]:
global_connection()

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [3]:
geno_dataset = Dataset.objects.get(file="ADAPTmap_genotypeTOP_20161201.zip")
phen_dataset = Dataset.objects.get(file="ADAPTmap_phenotype_20161201.zip")

In [4]:
print("\n".join(geno_dataset.contents))
print("\n".join(phen_dataset.contents))

ADAPTmap_genotypeTOP_20161201/
ADAPTmap_genotypeTOP_20161201/ADAPTmap_Breeds_20161201.csv
ADAPTmap_genotypeTOP_20161201/ADAPTmap_Breeds_20161201_fix.csv
ADAPTmap_genotypeTOP_20161201/binary_fileset/
ADAPTmap_genotypeTOP_20161201/binary_fileset/ADAPTmap_genotypeTOP_20161201.bed
ADAPTmap_genotypeTOP_20161201/binary_fileset/ADAPTmap_genotypeTOP_20161201.bim
ADAPTmap_genotypeTOP_20161201/binary_fileset/ADAPTmap_genotypeTOP_20161201.fam
ADAPTmap_genotypeTOP_20161201/binary_fileset/ADAPTmap_genotypeTOP_20161201.log
ADAPTmap_genotypeTOP_20161201/binary_fileset/ADAPTmap_genotypeTOP_20161201.nosex
ADAPTmap_phenotype_20161201/
ADAPTmap_phenotype_20161201/ADAPTmap_InfoSample_20161201.csv
ADAPTmap_phenotype_20161201/adaptmap_phenotypes_by_breed.xlsx


Data and metadata are splitted in two datasets. Moreover, by investigating on Adaptmap metadata (from ADAPTMAP project), I found that the same breed could be sampled in different countries, so is not possible to upload breed and aliases as I did until now. Take for example `ALP` breed for example:

In [5]:
datapath = phen_dataset.working_dir / "ADAPTmap_phenotype_20161201/ADAPTmap_InfoSample_20161201.csv"
data = pandas_open(datapath)
subset = data.loc[data['Breed_code']=='ALP', ['Breed_code', 'Sampling_Country']]
subset.groupby(['Breed_code', 'Sampling_Country']).size()

Breed_code  Sampling_Country
ALP         FRANCE               52
            ITALY               160
            SWITZERLAND          67
dtype: int64

This breed in particoular was samples in three different countries, `ITALY`, `SWITZERLAND`, `FRANCE`

How about the genotype files?

In [6]:
plink_path = geno_dataset.working_dir / "ADAPTmap_genotypeTOP_20161201/binary_fileset/ADAPTmap_genotypeTOP_20161201"
plink_file = plinkfile.open(str(plink_path))
sample_list = plink_file.get_samples()
locus_list = plink_file.get_loci()

Ok, try to get `ALP` samples, for example and get information on them

In [7]:
alp_samples = [sample for sample in sample_list if sample.fid == 'ALP']

In [8]:
sample_sex = [sample for sample in alp_samples if sample.sex != -9]

In [9]:
sample_sex

[]

Breed code matches fid in adaptmap dataset. However it seems to me I don't have the sex column in *plink* files, despite I could have such informations from metadata